In [1]:
from importlib.machinery import SourceFileLoader
import pandas as pd
import numpy  as np

In [2]:
soil_constants = [
     'Depth',
     'Porosity',
     'Bulk density',
     'Cation exchange capacity',
     'Soil sulfate adsorption capacity, half saturation',
     'Soil sulfate adsorption max capacity',
     'CO2 partial pressure',
     'Nitrification',
     'Denitrification',
     'Relative area',
     'SO4 weathering',
     'Ca weathering',
     'Mg weathering',
     'Na weathering',
     'K weathering',
     'Initial exchangeable Ca on soil as % of CEC',
     'Initial exchangeable Mg on soil as % of CEC',
     'Initial exchangeable Na on soil as % of CEC',
     'Initial exchangeable K on soil as % of CEC',
     '(log10) Al(OH)3 dissociation equilibrium constant',
     'Reduction in OA by SO4'
]

lake_constants = [
    'Depth',
    'Relative area', 
    'Reduction in OA by SO4'
]

long_term_parameters = ['Total Ca deposition',
                         'Total Mg deposition',
                         'Total Na deposition',
                         'Total K deposition',
                         'Total SO4 deposition',
                         'Total Cl deposition',
                         'Total NH4 deposition',
                         'Total NO3 deposition',
                         'Air temperature',
                         'Runoff',
                         'Observed Cl',
                         'Observed Ca',
                         'Observed Mg',
                         'Observed Na',
                         'Observed K',
                         'Observed SO4',
                         'Observed NH4',
                         'Observed NO3',
                         'Observed TOC (mgpl)',
                         'Observed TOC (mmolpl)',
                         'Observed TOC (mmolpm3)',
                         'Observed ANC',
                         'Observed lake pH',
                         'Observed soil pH',
                         'Observed ECa',
                         'Observed EMg',
                         'Observed ENa',
                         'Observed EK'
]

In [3]:
wr = SourceFileLoader("mobius", r"../../mobius.py").load_module()
#cu = SourceFileLoader("mobius_calib_uncert_lmfit", r"../../mobius_calib_uncert_lmfit.py").load_module()

multi = SourceFileLoader("multiloop", r"../multiloop.py").load_module()
magicopt = SourceFileLoader("magicopt",  r"../magicopt.py").load_module()

wr.initialize('../../../Applications/MAGIC/magic_forest.so')

parfile = 'templateparameters.dat'
infile  = 'templateinputs.dat'

# Read files
soil_constants_df = pd.read_excel('raw_data/CCE_soils_terndlakes_1995.xlsx')
lake_constants_df = pd.read_excel('raw_data/CCE_terndlakes_characteristics.xlsx')
long_term_parameters_df = pd.read_csv('raw_data/water_soil_chem_hydro_depos_fixed_to_match_magic.csv')

# Get unique station ids
station_ids = long_term_parameters_df['station_id'].unique()

# Get unique dates
dates = long_term_parameters_df['Date'].unique()


for station_id in station_ids:

    soil_station_df_filtered = soil_constants_df[soil_constants_df['station id'] == station_id]
    lake_station_df_filtered = lake_constants_df[lake_constants_df['station id'] == station_id]
    long_term_df_filtered = long_term_parameters_df[long_term_parameters_df['station_id'] == station_id]
  
    ds = wr.DataSet.setup_from_parameter_and_input_files(parfile, infile)
    
    # Assign soil constants
    for soil_constant in soil_constants:
        ds.set_parameter_double(soil_constant, ["Soil"], soil_station_df_filtered[soil_constant].iloc[0])

    for lake_constant in lake_constants:
        ds.set_parameter_double(lake_constant, ["Lake"], lake_station_df_filtered[lake_constant].iloc[0])

    for parameter in long_term_parameters:
        ds.set_input_series(parameter, [],  long_term_df_filtered[parameter].to_numpy())

    ds.write_parameters_to_file("initial_data/lake_%d.dat"%station_id)
    ds.write_inputs_to_file("inputs/lake_%d.dat"%station_id)
        
    ds.delete()
